In [33]:
import pandas as pd

# Load IoT sensor data from CSV (Generated in Homework 1)
df = pd.read_csv("IoT_Data_Simulation_Homework.csv")

# Display the first few rows
print(df.head())

    Sensor ID                                     Location  Temperature (°C)  \
0     Arm_968  Haleyberg, Saint Vincent and the Grenadines             32.88   
1   Serve_918                          Lopezland, Cambodia             30.65   
2   Every_472                        Theodorebury, Eritrea             25.67   
3  Forget_663                     New Clarence, Cape Verde             30.05   
4    Safe_507                       East Justinmouth, Oman             25.26   

   Humidity (%)  Air Quality Index (AQI)            Timestamp  soil_moisture  \
0            49                      179  2025-05-10 00:27:50             16   
1            77                       75  2025-05-10 00:27:51             27   
2            41                      170  2025-05-10 00:27:52             22   
3            47                       72  2025-05-10 00:27:53             26   
4            78                      176  2025-05-10 00:27:54             25   

   pm25  pm10        o3   no2  so2    

In [23]:
from web3 import Web3

ganache_url = "http://127.0.0.1:7545"  # Change port if necessary
web3 = Web3(Web3.HTTPProvider(ganache_url))

if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")


✅ Connected to Ganache successfully!


In [24]:
import json

contract_address = "0x909508710824370119b92d2F0B34575B0d81E8CD"

with open('abi.json') as f:
	abi = json.load(f)  # Replace with actual ABI JSON list

# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=abi)


# Set the default sender address (first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]


print(f"✅ Connected to Smart Contract at {contract_address}")


✅ Connected to Smart Contract at 0x909508710824370119b92d2F0B34575B0d81E8CD


In [25]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total Records: {total_records}")

Total Records: 0


In [26]:
txn = contract.functions.storeData("TEST001", "Temperature", "22.5°C").transact({
    'from': web3.eth.default_account,
    'gas': 1000000
})
web3.eth.wait_for_transaction_receipt(txn)
print("✅ Dummy data stored on blockchain!")


✅ Dummy data stored on blockchain!


In [27]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total Records: {total_records}")

record = contract.functions.getRecord(0).call()
print("First Stored Record:", record)


Total Records: 1
First Stored Record: [1748634911, 'TEST001', 'Temperature', '22.5°C']


In [28]:
import time

def send_iot_data(device_id, data_type, data_value):
    txn = contract.functions.storeData(device_id, data_type, data_value).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Data Stored: {data_type} - {data_value}, Txn Hash: {receipt.transactionHash.hex()}")


In [29]:

sensor_fields = [
    'Temperature (°C)', 'Humidity (%)', 'Air Quality Index (AQI)',
    'soil_moisture', 'pm25', 'pm10', 'o3', 'no2', 'so2', 'co',
    'water_quality', 'ph', 'turbidity', 'contamination'
]

# Only send the first 50 rows for testing
for i, row in enumerate(df.iterrows()):
    if i >= 50:
        break
    row = row[1]
    sensor_id = str(row["Sensor ID"])
    for field in sensor_fields:
        try:
            data_value = str(row[field])
            send_iot_data(sensor_id, field, data_value)
            time.sleep(1)
        except Exception as e:
            print(f"❌ Error sending {field} for {sensor_id}: {e}")

✅ Data Stored: Temperature (°C) - 32.88, Txn Hash: d36710a9d9aa9df8eed86352c0a527fc63249f8e38a25a4e4c8e8bf19b56ce6a
✅ Data Stored: Humidity (%) - 49, Txn Hash: be200024ce362d03d83f5161948c58d3269a10797358c1785d803e6285e847c5
✅ Data Stored: Air Quality Index (AQI) - 179, Txn Hash: 4f31fa9d59bb2845ffbcb358f82a861fd386298fe532602c261964abdce72dcf
✅ Data Stored: soil_moisture - 16, Txn Hash: d69c8d720b5af714168ffb09fa8b6a9998cf5b5495a23d7acc3d58a1eead3bc7
✅ Data Stored: pm25 - 479, Txn Hash: 3122bcfac3795cab18a6897ddce022d1dc2ffd9b827ed9626e9be3048d5ccd73
✅ Data Stored: pm10 - 237, Txn Hash: 871322b9609f0ed891bc2813341410c854086c2146666991b089f855e3a4b36b
✅ Data Stored: o3 - 0.1049158253256662, Txn Hash: 9022d068fa51fb6394e49d78e82a212422c2a5890e91ffa36496229620c8c459
✅ Data Stored: no2 - 391, Txn Hash: b31c95e877de192c5d7a6c74195e7144aa8900d4cf78fd9eba348dd3fb9387dd
✅ Data Stored: so2 - 537, Txn Hash: 3acb888077eba0c06a575916cdfb8bd4ddff5e814eef783943b8bceba7af729e
✅ Data Stored: co - 43.

In [30]:
total_records = contract.functions.getTotalRecords().call()
print(f"📦 Total IoT records stored: {total_records}")

📦 Total IoT records stored: 701


In [31]:

if total_records > 0:
    record = contract.functions.getRecord(0).call()
    print("📝 First Stored Record:", record)

📝 First Stored Record: [1748634911, 'TEST001', 'Temperature', '22.5°C']
